# Temperature-dependent solvation free energy and vapor-liquid equilibrium calculations

This ipython notebook calculates temperature-dependent solvation free energy and vapor-liquid equilibrium ratio for a dilue binary mixture at the saturation pressure of the solvent. Read documentation on solvation thermochemistry for detailed method (http://reactionmechanismgenerator.github.io/RMG-Py/users/rmg/liquids.html?highlight=liquid). Please cite our work in your publication:
    
    Chung, Y. & Gillis R. & Green W. H. (2020). Temperature dependent solvation free energy estimation from
    minimal data. Manuscript submitted for publication. 

Code written by Yunsie Chung.

In [ ]:
import os.path
import math
import numpy as np
import matplotlib.pyplot as plt
from CoolProp.CoolProp import PropsSI

from rmgpy import settings
from rmgpy.rmg.main import Species
from rmgpy.data.solvation import SolvationDatabase

%matplotlib inline

solvationDatabase = SolvationDatabase()
solvationDatabase.load(os.path.join(settings['database.directory'], 'solvation'))

# Initial setup to get the available solvent list and valid tempertaure range

In [ ]:
print("Available solvents and their valid temperature ranges for T-dependent solvation calculation are:\n")

for index, entry in solvationDatabase.libraries['solvent'].entries.items():
    if entry.data.s_h is not None and entry.data.name_in_coolprop is not None:
        Tc = "%.2f" % entry.data.get_solvent_critical_temperature()
        print('     - ' + index + ': 280 K - {0} K'.format(Tc))

# User Input: Solute SMILES, Solvent Name, and Temperature

In [ ]:
################# Change these as needed ##########################
solute_smiles = 'CC(C)O'            # most of solutes that contain only H, C, N, O, S are available
solvent_name = 'water'              # this should one of the solvent name from the list above
temprature = [300, 400, 500, 600]   # give temperature(s) in K. This should be within the valid range shown above
###################################################################

# Note: if you have your own Abraham parameters for the solute, add it to 
# RMG-database/input/solvation/libraries/solute.py as a new entry, and re-run this ipython notebook.

# Run solvation calculations

In [ ]:
solute = Species().from_smiles(solute_smiles)
solute.generate_resonance_structures()
solute_data = solvationDatabase.get_solute_data(solute)
solvent_data = solvationDatabase.get_solvent_data(solvent_name)

print('Results are given in K-factor (VLE solute mole fraction ratio, y2/x2) and solvation free energy:\n')
for T in temprature:
    Kfactor = solvationDatabase.get_Kfactor(solute_data, solvent_data, T)
    dGsolv = solvationDatabase.get_T_dep_solvation_energy(solute_data, solvent_data, T) / 1000 # in kJ/mol
    print('     At {0} K, K-factor = {1:.3f}, solvation free energy = {2:.2f} kJ/mol'.format(T, Kfactor, dGsolv))

# Make useful plots from 280 K up to the critical point of the solvent

In [ ]:
Tc = solvent_data.get_solvent_critical_temperature() # critical temperature of the solvent in K
temp_list = np.linspace(280, Tc-0.01, 100)

log_Kfactor_list = []
log_KfactorPsat_list = []
dGsolv_list = []

for T in temp_list:
    Kfactor = solvationDatabase.get_Kfactor(solute_data, solvent_data, T)
    dGsolv = solvationDatabase.get_T_dep_solvation_energy(solute_data, solvent_data, T) / 1000 # in kJ/mol
    Psat = PropsSI('P', 'T', T, 'Q', 0, solvent_data.name_in_coolprop) # saturation pressure of the solvent, in Pa
    log_Kfactor_list.append(math.log(Kfactor))
    log_KfactorPsat_list.append(math.log(Kfactor*Psat*1e-6)) # in ln(1/MPa)
    dGsolv_list.append(dGsolv)

In [ ]:
fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(2, 2, 1)
plt.plot(temp_list, log_Kfactor_list)
plt.title('ln(K-factor)', fontsize=20)
ax.set_xlabel('Temperature (K)', fontsize = 16)
ax.set_ylabel('ln($K_{2,1}^{\infty}$)', fontsize = 16)

ax = fig.add_subplot(2, 2, 2)
plt.plot(temp_list, log_KfactorPsat_list)
plt.title('ln(K-factor * $P^{sat}$)\n$P^{sat}$ = saturation pressure of the solvent', fontsize=20)
ax.set_xlabel('Temperature (K)', fontsize = 16)
ax.set_ylabel('ln($K_{2,1}^{\infty}P^{sat}$/1 MPa)', fontsize = 16)

ax = fig.add_subplot(2, 2, 3)
plt.plot(temp_list, dGsolv_list)
plt.title('Solvation Gibbs Free Energy', fontsize=20)
ax.set_xlabel('Temperature (K)', fontsize = 16)
ax.set_ylabel('Solvation Energy (kJ/mol)', fontsize = 16)
